## Minimum Cost Network Flow Problem (MCNFP)

Consider the network in the following image:

<img src="./transport_network.png" alt="Alternative text" />

Here, $V = \{1,2,3,4\}$ is the set of nodes, and $E=\{(1,2),(1,3),(2,3),(3,2),(2,4),(3,4)\}$ is the set of edges. 
The numbers next to the nodes are net supplies (e.g., the net supply for node $1$ is $10$, and net supply for node $4$ is $-50$). 

On each of the edges, the first number is the unit cost and the interval gives flow bounds. 
For example on edge $(1,3)$ the unit cost is $1$, the lower bound of flow is $0$, and the upper bound is $\infty$.

In [21]:
#Number of vertices
n = 4

#1 where in entries with edges, 0 otherwise. For example, edge from 1->2 means we place
#a 1 in entry 1,2 
edgeset = [0 1 1 0;
           0 0 1 1;
           0 1 0 1;
           0 0 0 0]

costs = [0 5 1 0;
         0 0 1 2;
         0 10 0 8;
         0 0 0 0]
upper = [0 30 Inf 0;
         0 0 Inf Inf;
         0 10 0 20;
         0 0 0 0]
supply = [10 30 10 -50]
;

In [22]:
using JuMP, HiGHS

mcnfp = Model(HiGHS.Optimizer)

#Create variables for each edge.
@variable(mcnfp, upper[i,j] >= x[i in 1:numvertices, j in 1:n] >= 0)

#Set all non-existent edges to 0
@constraint(mcnfp, removeedges[i = 1:n, j = 1:n; edgeset[i, j] == 0], x[i,j] == 0)

#Flow constraint
@constraint(mcnfp, flow[i in 1:n], sum(x[i,j] for j in 1:n) - sum(x[j,i] for j in 1:n) == supply[i])

@objective(mcnfp, Min, sum(sum(costs[i,j]*x[i,j] for i in 1:n) for j in 1:n))

print(mcnfp)

In [23]:
optimize!(mcnfp)

Presolving model
4 rows, 6 cols, 12 nonzeros
1 rows, 4 cols, 4 nonzeros
1 rows, 4 cols, 4 nonzeros
Presolve : Reductions: rows 1(-13); columns 4(-12); elements 4(-30)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1     2.1000000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  2.1000000000e+02
HiGHS run time      :          0.00


Alternative approach: use the incidence matrix $A$.

In [33]:
#Directly create A
A = [1  1  0  0  0  0
    -1  0  1  1 -1  0
     0 -1 -1  0  1  1
     0  0  0 -1  0 -1]
numedges = 6

#Or programatically generate it from edgeset
#numedges = sum(sum(edgeset[i,j] for i in 1:n) for j in 1:n)
#A = zeros(n,numedges)
#edgecount = 1
#for i in 1:n
#    for j in 1:n
#        if(edgeset[i,j] == 1)
#            A[i, edgecount] = 1
#            A[j, edgecount] = -1
#            edgecount = edgecount+1
#        end
#    end
#end


costs = [5 1 1 2 10 8]
upper = [30 Inf Inf Inf 10 20]
supply = [10; 30; 10; -50]
;

In [34]:
using JuMP, HiGHS

mcnfp2 = Model(HiGHS.Optimizer)

#Create variables for each edge.
@variable(mcnfp2, upper[i] >= x[i in 1:numedges] >= 0)
#Flow constraint
@constraint(mcnfp2, flow, A*x .== supply)

@objective(mcnfp2, Min, sum(costs[i]*x[i] for i in 1:numedges))

print(mcnfp2)

In [35]:
optimize!(mcnfp2)

Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No invertible representation for getDualRay
